In [ ]:
# specs_path = './../../../../GAMADR3_SPECTRA/DATABASE/'
# specs_list = os.listdir(spec_path)

# Libraries

In [1]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import astropy.io.fits   as pf
import os

# Dataset

In [2]:
dataset_path = './../../Catalogue/Match08_smallemlines_inputcattype.csv'
my_data      = pd.read_csv(dataset_path)

# Plot the distributions of S/N

In [39]:
whan_class = my_data['WHAN_CLASS'].values.astype(str)
uv_class   = my_data['UV_CLASS_YI2011'].values.astype(str)
cataid     = my_data['CATAID'].values.astype(str)
specid     = my_data['SPECID'].values.astype(str)
s2n        = my_data['S2N'].values.astype(float)
chi2_gama  = my_data['CHI2'].values.astype(float)
redshift   = my_data['Z'].values.astype(float)

In [37]:
uv_unique   = np.unique(uv_class, return_counts=True)
whan_unique = np.unique(whan_class, return_counts=True)
print (uv_unique, '\n', whan_unique)

(array(['RSF', 'UV_UPTURN', 'UV_WEAK'], dtype='<U9'), array([13840,   210,   296])) 
 (array(['Retired/Passive', 'SF', 'nan', 'sAGN', 'wAGN'], dtype='<U15'), array([ 876, 9408, 1285, 2184,  593]))


In [35]:
idxs = []
idx_uvup_rp = np.where((uv_class=='UV_UPTURN')*(whan_class=='Retired/Passive'))

In [42]:
# print (redshift.max(), redshift.min())

### Plot settings (yes, I'm lazy)

In [ ]:
bins = 50
palette = []